In [1]:
include("../src/Julia.jl")

objc[852]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[852]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[852]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[852]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/ipm_grid3_8000_all/";

In [4]:
graphs = readdir(gpath);
len = length(graphs)
graphs = graphs[ceil(Int64,len * 0.4)+1:ceil(Int64,len * 0.6)];

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
#         x = readFromFile(path * "x.vec");
        x = zeros(a.n)
        b = readFromFile(path * "b.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        bdef = copy(b)
        bdef = bdef - mean(bdef);
        
        numIts = 100;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(bdef);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, bdef, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - bdef) / norm(bdef)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now using higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                
                xHP = zeros(BigFloat, a.n)
                
                bHP = toHighPrecision(b; precision=hp);
                bHP = bHP - mean(bHP)
                
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.461553 seconds (639.99 k allocations: 29.152 MB, 1.45% gc time)
  0.169505 seconds (247.53 k allocations: 6.748 MB, 5.14% gc time)
  0.915776 seconds (15.85 M allocations: 389.480 MB, 5.23% gc time)
 50.765296 seconds (262.31 M allocations: 6.633 GB, 36.29% gc time)
 57.126850 seconds (262.14 M allocations: 7.268 GB, 37.76% gc time)
 63.407288 seconds (262.14 M allocations: 8.552 GB, 36.83% gc time)
 74.717853 seconds (262.14 M allocations: 11.121 GB, 35.98% gc time)
 87.191977 seconds (262.14 M allocations: 16.260 GB, 34.97% gc time)
  0.017986 seconds (40.19 k allocations: 4.565 MB)
  0.004829 seconds (155.23 k allocations: 2.858 MB)
  0.779255 seconds (15.68 M allocations: 380.548 MB, 25.10% gc time)
 52.427281 seconds (262.18 M allocations: 6.626 GB, 37.30% gc time)
 57.954288 seconds (262.18 M allocations: 7.268 GB, 37.71% gc time)
 63.381597 seconds (262.18 M allocations: 8.553 GB, 36.51% gc time)
 73.146485 seconds (262.18 M allocations: 11.122 GB, 36.18% gc time)
 87.692754